In [39]:

import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp

In [3]:
with open("alldata.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))
print(data.keys())

name_list = data['name_list']
industry_list = data['industry_list']
index_list = data['index_list']
cusip_list = data['cusip_list']
universe = data['universe']

<class 'dict'>
dict_keys(['name_list', 'industry_list', 'index_list', 'cusip_list', 'universe'])


In [4]:
universe

,RIC,name,date,index,ISIN Code,TRBC Industry,CUSIPE
0,1COVG.DE,COVESTRO ORD,2019-12-31,.GDAXI,DE0006062144,Chemicals,D15349109
1,AALB.AS,AALBERTS ORD,2019-12-31,.AEX,NL0000852564,"Machinery, Tools, Heavy Vehicles, Trains & Ships",N00089271
2,ABB.ST,ABB ORD,2019-12-31,.OMXS30,CH0012221716,"Machinery, Tools, Heavy Vehicles, Trains & Ships",H0010V101
3,ABI.BR,ANHEUSER-BUSCH INBEV SA/NV ORD,2019-12-31,.BFX,BE0974293251,Beverages,B639CJ108
4,ABNd.AS,ABN AMRO BANK ORD,2019-12-31,.AEX,NL0011540547,Banking Services,N0162C102
...,...,...,...,...,...,...,...
1957,WLSNc.AS,WOLTERS KLUWERS ORD,2024-12-31,.AEX,NL0000395903,Professional & Commercial Services,N9643A197
1958,WRT1V.HE,WARTSILA ORD,2024-12-31,.OMXH25,FI0009003727,"Machinery, Tools, Heavy Vehicles, Trains & Ships",X98155116
1959,YAR.OL,YARA INTL ORD,2024-12-31,.OBX,NO0010208051,Chemicals,R9900C106
1960,ZALG.DE,ZALANDO ORD SHS,2024-12-31,.GDAXI,DE000ZAL1111,Software & IT Services,D98423102


In [40]:
rd.open_session()

ric = "ABI.BR"
start_date = "2019-09-01"
end_date = "2025-08-01"

price_df = rd.get_history(
    universe=ric,
    start=start_date,
    end=end_date,
    interval="daily"
)

df = pd.DataFrame({
    'price': price_df['TRDPRC_1'],
    'volume_shares': price_df['ACVOL_UNS'],
    'bid': price_df['BID'],
    'ask': price_df['ASK']
})

fundamental_response = rd.get_data(
    universe=ric,
    fields=[
        "TR.TotalReturn.Date",
        "TR.TotalReturn",           # Total Return Index
        "TR.PriceToBook",           # Price to Book (Market-to-Book)
        "TR.CompanyMarketCap"       # Market Cap
    ],
    parameters={
        "SDate": start_date,
        "EDate": end_date,
        "Frq": "D",
        "Curn": "EUR"
    }
)

print("\nFundamental data response:")
print(fundamental_response.head(10))
print(f"\nColumns: {fundamental_response.columns.tolist()}")

if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
    fund_df = fundamental_response.copy()
    
    if 'Date' in fund_df.columns:
        fund_df['Date'] = pd.to_datetime(fund_df['Date'])
        fund_df.set_index('Date', inplace=True)
        
        if 'Instrument' in fund_df.columns:
            fund_df.drop('Instrument', axis=1, inplace=True)
        
        print(f"\nFundamental data with Date index:")
        print(fund_df.head(10))
        
        df = df.join(fund_df, how='left')
    else:
        print("\nWARNING: No Date column found in fundamental data!")
        print("Available columns:", fund_df.columns.tolist())

column_mapping = {
    'Total Return': 'tri',
    'Price To Book Value': 'mtbv',
    'Company Market Cap': 'cap'
}

for old_name, new_name in column_mapping.items():
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)

df['volume'] = df['volume_shares'] * df['price']
df.drop('volume_shares', axis=1, inplace=True)

if 'mtbv' in df.columns:
    df['mtbv'] = df['mtbv'].ffill()
if 'cap' in df.columns:
    df['cap'] = df['cap'].ffill()

desired_order = ['price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
existing_cols = [col for col in desired_order if col in df.columns]
df = df[existing_cols]

print("\n" + "="*80)
print("FINAL DATASET")
print("="*80)
print(df.head(15))
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nNull counts:\n{df.isnull().sum()}")
print(f"\nSample with non-null data:")
print(df[df['tri'].notna()].head(10))

df.to_csv("ABI_BR_Data.csv", index=True)
print(f"\nData saved to ABI_BR_Data.csv")

rd.close_session()


Fundamental data response:
  Instrument       Date  Total Return   Company Market Cap
0     ABI.BR 2025-08-01     -2.840467  146008271111.880005
1     ABI.BR 2025-07-31    -11.562285  145517230886.640015
2     ABI.BR 2025-07-30      0.832755  145060055504.519989
3     ABI.BR 2025-07-29      0.348189  143840921152.199005
4     ABI.BR 2025-07-28     -3.624161  143078962181.998993
5     ABI.BR 2025-07-25     -0.633545  145720419945.359009
6     ABI.BR 2025-07-24      0.671366  144162637161.839996
7     ABI.BR 2025-07-23       1.08585  144619812543.959991
8     ABI.BR 2025-07-22      0.408859  148835985512.399994
9     ABI.BR 2025-07-21      0.548133  147633783581.639008

Columns: ['Instrument', 'Date', 'Total Return', 'Company Market Cap']

Fundamental data with Date index:
            Total Return   Company Market Cap
Date                                         
2025-08-01     -2.840467  146008271111.880005
2025-07-31    -11.562285  145517230886.640015
2025-07-30      0.832755  1450600

In [36]:
import time

rd.open_session()

unique_rics = universe['RIC'].unique()
print(f"Total rows in universe: {len(universe)}")
print(f"Unique RICs to process: {len(unique_rics)}")

start_date = "2019-09-01"
end_date = "2025-08-01"

all_data = []
failed_rics = []

total_rics = len(unique_rics)
print(f"\nProcessing {total_rics} unique stocks...")

for idx, ric in enumerate(unique_rics, 1):
    print(f"[{idx}/{total_rics}] Processing {ric}...", end=' ')
    
    try:
        # --- Fetch historical PRICING data ---
        price_df = rd.get_history(
            universe=ric,
            start=start_date,
            end=end_date,
            interval="daily"
        )
        
        # Extract only what we need
        df = pd.DataFrame({
            'RIC': ric,  # Add RIC column
            'price': price_df['TRDPRC_1'],
            'volume_shares': price_df['ACVOL_UNS'],
            'bid': price_df['BID'],
            'ask': price_df['ASK']
        })
        
        # --- Fetch FUNDAMENTAL data WITH DATES ---
        fundamental_response = rd.get_data(
            universe=ric,
            fields=[
                "TR.TotalReturn.Date",
                "TR.TotalReturn",
                "TR.PriceToBook",
                "TR.CompanyMarketCap"
            ],
            parameters={
                "SDate": start_date,
                "EDate": end_date,
                "Frq": "D",
                "Curn": "EUR"
            }
        )
        
        # Process fundamental data
        if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
            fund_df = fundamental_response.copy()
            
            if 'Date' in fund_df.columns:
                fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                fund_df.set_index('Date', inplace=True)
                
                if 'Instrument' in fund_df.columns:
                    fund_df.drop('Instrument', axis=1, inplace=True)
                
                # Merge with price data
                df = df.join(fund_df, how='left')
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        df['volume'] = df['volume_shares'] * df['price']
        df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill mtbv and cap
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        # Append to list
        all_data.append(df)
        
        print(f"✓ {len(df)} rows")
        
        # Rate limiting - sleep briefly to avoid overwhelming the API
        if idx % 10 == 0:
            time.sleep(2)
        
    except Exception as e:
        print(f"✗ Failed: {e}")
        failed_rics.append(ric)
        continue

# --- 6. Combine all data ---
print("\n" + "="*80)
print("PROCESSING COMPLETE")
print("="*80)
print(f"Successfully processed: {len(all_data)} stocks")
print(f"Failed: {len(failed_rics)} stocks")

if failed_rics:
    print(f"\nFailed RICs:")
    for ric in failed_rics:
        print(f"  - {ric}")

# --- 7. Create combined DataFrame ---
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"\nCombined dataset shape: {combined_df.shape}")
    print(f"Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
    print(f"Unique stocks: {combined_df['RIC'].nunique()}")
    
    # Display sample
    print("\nSample data:")
    print(combined_df.head(10))
    
    # Save to CSV
    combined_df.to_csv("all_stocks_data.csv", index=False)
    print(f"\n✓ Saved to all_stocks_data.csv")
    
    # Show null counts
    print("\nNull counts by column:")
    print(combined_df.isnull().sum())
else:
    print("\nNo data retrieved!")

# --- 8. Close session ---
rd.close_session()

print("\nDone!")

Total rows in universe: 1962
Unique RICs to process: 411

Processing 411 unique stocks...
[1/411] Processing 1COVG.DE... 

d:\myenv\Lib\site-packages\refinitiv\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✗ Failed: 'TRDPRC_1'
[2/411] Processing AALB.AS... 

d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1057:RuntimeWarning: invalid value encountered in cast
d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1081:RuntimeWarning: invalid value encountered in cast


✓ 1517 rows
[3/411] Processing ABB.ST... 

...

✓ 1488 rows
[410/411] Processing WAWI.OL... 

d:\myenv\Lib\site-packages\refinitiv\data\_tools\_dataframe.py:177:FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✗ Failed: 'TRDPRC_1'
[411/411] Processing ZELA.CO... 

d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1057:RuntimeWarning: invalid value encountered in cast
d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1081:RuntimeWarning: invalid value encountered in cast
d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1057:RuntimeWarning: invalid value encountered in cast
d:\myenv\Lib\site-packages\pandas\core\dtypes\cast.py:1081:RuntimeWarning: invalid value encountered in cast


✓ 1481 rows

PROCESSING COMPLETE
Successfully processed: 323 stocks
Failed: 88 stocks

Failed RICs:
  - 1COVG.DE
  - ADSGn.DE
  - AKER.OL
  - AKRBP.OL
  - ALVG.DE
  - BAKKA.OL
  - BASFn.DE
  - BAYGn.DE
  - BEIG.DE
  - BMWG.DE
  - BWLPG.OL
  - BWO.OL
  - CONG.DE
  - DB1Gn.DE
  - DBKGn.DE
  - DHLn.DE
  - DNB.OL
  - DNO.OL
  - DTEGn.DE
  - EONGn.DE
  - EQNR.OL
  - FMEG.DE
  - FREG.DE
  - FRO.OL
  - GJFG.OL
  - GOGL.OL^H25
  - HEIG.DE
  - HNKG_p.DE
  - IFXGn.DE
  - LHAG.DE
  - LIN.DE
  - LSG.OL
  - MBGn.DE
  - MOWI.OL
  - MRCG.DE
  - MTXGn.DE
  - MUVGn.DE
  - NAS.OL
  - NEL.OL
  - NHY.OL
  - ORK.OL
  - RWEG.DE
  - SALM.OL
  - SAPG.DE
  - SIEGn.DE
  - STB.OL
  - SUBC.OL
  - TEL.OL
  - TGS.OL
  - TOM.OL
  - VENDA.OL
  - VNAn.DE
  - VOWG_p.DE
  - WDIG.DE^A21
  - YAR.OL
  - DHER.DE
  - DWNG.DE
  - ENTRA.OL
  - SCATC.OL
  - ADEA.OL^F24
  - AIRG.DE
  - BNRGn.DE
  - ENR1n.DE
  - HFGG.DE
  - KAHOT.OL^A24
  - MPCC.OL
  - NOD.OL
  - PSHG_p.DE
  - PUMG.DE
  - QIA.DE
  - RECSI.OL
  - SATG_p.DE
  - SHL

In [37]:
missing = set(universe['RIC'].unique()) - set(pd.read_csv("all_stocks_data.csv")['RIC'].unique())
print(f"Missing: {len(missing)} RICs")
print(sorted(missing))

Missing: 88 RICs
['1COVG.DE', 'ADEA.OL^F24', 'ADSGn.DE', 'AIRG.DE', 'AKER.OL', 'AKRBP.OL', 'ALVG.DE', 'AUTO.OL', 'BAKKA.OL', 'BASFn.DE', 'BAYGn.DE', 'BEIG.DE', 'BMWG.DE', 'BNRGn.DE', 'BORR.OL^A25', 'BWLPG.OL', 'BWO.OL', 'CBKG.DE', 'CONG.DE', 'DB1Gn.DE', 'DBKGn.DE', 'DHER.DE', 'DHLn.DE', 'DNB.OL', 'DNO.OL', 'DTEGn.DE', 'DTGGe.DE', 'DWNG.DE', 'ENR1n.DE', 'ENTRA.OL', 'EONGn.DE', 'EQNR.OL', 'FMEG.DE', 'FREG.DE', 'FRO.OL', 'GJFG.OL', 'GOGL.OL^H25', 'HAFNI.OL', 'HAUTO.OL', 'HEIG.DE', 'HFGG.DE', 'HNKG_p.DE', 'HNRGn.DE', 'IFXGn.DE', 'KAHOT.OL^A24', 'KOG.OL', 'LHAG.DE', 'LIN.DE', 'LSG.OL', 'MBGn.DE', 'MOWI.OL', 'MPCC.OL', 'MRCG.DE', 'MTXGn.DE', 'MUVGn.DE', 'NAS.OL', 'NEL.OL', 'NHY.OL', 'NOD.OL', 'ORK.OL', 'P911_p.DE', 'PGS.OL^G24', 'PSHG_p.DE', 'PUMG.DE', 'QIA.DE', 'RECSI.OL', 'RHMG.DE', 'RWEG.DE', 'SALM.OL', 'SAPG.DE', 'SATG_p.DE', 'SCATC.OL', 'SHLG.DE', 'SIEGn.DE', 'STB.OL', 'SUBC.OL', 'SY1G.DE', 'TEL.OL', 'TGS.OL', 'TOM.OL', 'VAR.OL', 'VENDA.OL', 'VNAn.DE', 'VOWG_p.DE', 'WAWI.OL', 'WDIG.DE^A

In [43]:
df = pd.read_csv("all_stocks_data.csv")

myday = df[['Date']].drop_duplicates().sort_values('Date')
myday.to_csv("myday.csv", index=False)

In [44]:
combined_df = pd.read_csv("all_stocks_data.csv")

print("Analyzing existing data for incomplete RICs...")
print("="*80)
print(f"Columns in CSV: {combined_df.columns.tolist()}")
print(f"Shape: {combined_df.shape}")

# --- Find RICs with ANY missing data ---
# Only check columns that actually exist
columns_to_check = ['price', 'tri', 'volume', 'cap', 'bid', 'ask']
existing_columns = [col for col in columns_to_check if col in combined_df.columns]

print(f"\nChecking these columns for missing data: {existing_columns}")

agg_dict = {col: lambda x: x.isnull().sum() for col in existing_columns}

rics_with_missing = combined_df.groupby('RIC').agg(agg_dict).reset_index()

# Calculate total missing per RIC
rics_with_missing['total_missing'] = rics_with_missing[existing_columns].sum(axis=1)

# Filter to only RICs with ANY missing data
incomplete_rics = rics_with_missing[rics_with_missing['total_missing'] > 0]['RIC'].tolist()

print(f"\nRICs with complete data: {len(rics_with_missing) - len(incomplete_rics)}")
print(f"RICs with missing data: {len(incomplete_rics)}")

if len(incomplete_rics) == 0:
    print("\nNo incomplete RICs found! All data is complete.")
else:
    print(f"\nIncomplete RICs: {incomplete_rics[:20]}")  # Show first 20
    if len(incomplete_rics) > 20:
        print(f"... and {len(incomplete_rics) - 20} more")
    
    # Show summary of what's missing
    print("\nMissing data breakdown for incomplete RICs:")
    print(rics_with_missing[rics_with_missing['total_missing'] > 0].sort_values('total_missing', ascending=False).head(20))

# --- Retry all incomplete RICs ---
print("\n" + "="*80)
print(f"RETRYING {len(incomplete_rics)} INCOMPLETE RICs")
print("="*80)

rd.open_session()

retry_data = []
retry_failed = {}

start_date = "2019-09-01"
end_date = "2025-08-01"

for idx, ric in enumerate(incomplete_rics, 1):
    print(f"[{idx}/{len(incomplete_rics)}] Processing {ric}...", end=' ')
    
    try:
        # --- Fetch historical PRICING data ---
        price_df = rd.get_history(
            universe=ric,
            start=start_date,
            end=end_date,
            interval="daily"
        )
        
        # Extract only what we need
        df = pd.DataFrame({
            'RIC': ric,
            'price': price_df['TRDPRC_1'] if 'TRDPRC_1' in price_df.columns else None,
            'volume_shares': price_df['ACVOL_UNS'] if 'ACVOL_UNS' in price_df.columns else None,
            'bid': price_df['BID'] if 'BID' in price_df.columns else None,
            'ask': price_df['ASK'] if 'ASK' in price_df.columns else None
        })
        
        # --- Fetch FUNDAMENTAL data WITH DATES ---
        try:
            fundamental_response = rd.get_data(
                universe=ric,
                fields=[
                    "TR.TotalReturn.Date",
                    "TR.TotalReturn",
                    "TR.PriceToBook",
                    "TR.CompanyMarketCap"
                ],
                parameters={
                    "SDate": start_date,
                    "EDate": end_date,
                    "Frq": "D",
                    "Curn": "EUR"
                }
            )
            
            # Process fundamental data
            if fundamental_response is not None and isinstance(fundamental_response, pd.DataFrame):
                fund_df = fundamental_response.copy()
                
                if 'Date' in fund_df.columns:
                    fund_df['Date'] = pd.to_datetime(fund_df['Date'])
                    fund_df.set_index('Date', inplace=True)
                    
                    if 'Instrument' in fund_df.columns:
                        fund_df.drop('Instrument', axis=1, inplace=True)
                    
                    # Merge with price data
                    df = df.join(fund_df, how='left')
        
        except Exception as fund_error:
            pass  # Continue even if fundamentals fail
        
        # Rename columns
        column_mapping = {
            'Total Return': 'tri',
            'Price To Book Value': 'mtbv',
            'Company Market Cap': 'cap'
        }
        
        for old_name, new_name in column_mapping.items():
            if old_name in df.columns:
                df.rename(columns={old_name: new_name}, inplace=True)
        
        # Compute volume in EUR
        if 'volume_shares' in df.columns and 'price' in df.columns:
            df['volume'] = df['volume_shares'] * df['price']
            df.drop('volume_shares', axis=1, inplace=True)
        
        # Forward-fill mtbv and cap
        if 'mtbv' in df.columns:
            df['mtbv'] = df['mtbv'].ffill()
        if 'cap' in df.columns:
            df['cap'] = df['cap'].ffill()
        
        # Reset index to make Date a column
        df.reset_index(inplace=True)
        if 'index' in df.columns:
            df.rename(columns={'index': 'Date'}, inplace=True)
        
        # Reorder columns - only include columns that exist
        desired_order = ['RIC', 'Date', 'price', 'tri', 'volume', 'mtbv', 'cap', 'bid', 'ask']
        existing_cols = [col for col in desired_order if col in df.columns]
        df = df[existing_cols]
        
        # Append to list
        retry_data.append(df)
        
        print(f"✓ {len(df)} rows")
        
        # Rate limiting
        if idx % 10 == 0:
            time.sleep(2)
        
    except Exception as e:
        print(f"✗ Failed: {e}")
        retry_failed[ric] = str(e)
        continue

rd.close_session()

# --- Save retry results ---
print("\n" + "="*80)
print("RETRY COMPLETE")
print("="*80)

if retry_data:
    attempt2_df = pd.concat(retry_data, ignore_index=True)
    
    print(f"Successfully re-retrieved: {len(retry_data)} RICs")
    print(f"Failed on retry: {len(retry_failed)} RICs")
    print(f"Total rows in attempt2: {len(attempt2_df)}")
    print(f"Date range: {attempt2_df['Date'].min()} to {attempt2_df['Date'].max()}")
    
    # Check if data is still incomplete
    print("\nMissing data in attempt2:")
    print(attempt2_df.isnull().sum())
    
    # Save to CSV
    attempt2_df.to_csv("attempt2.csv", index=False)
    print(f"\n✓ Saved to attempt2.csv")
    
    # Show which RICs still have issues
    rics_still_incomplete = attempt2_df.groupby('RIC').apply(
        lambda x: x.isnull().sum().sum()
    )
    rics_still_incomplete = rics_still_incomplete[rics_still_incomplete > 0]
    
    if len(rics_still_incomplete) > 0:
        print(f"\nWarning: {len(rics_still_incomplete)} RICs still have missing data after retry:")
        print(rics_still_incomplete.sort_values(ascending=False).head(20))
else:
    print("\nNo data retrieved on retry!")

if retry_failed:
    print(f"\nFailed RICs:")
    for ric, error in list(retry_failed.items())[:10]:
        print(f"  {ric}: {error[:80]}")
    
    # Save failed list
    pd.DataFrame([
        {'RIC': ric, 'Error': error} 
        for ric, error in retry_failed.items()
    ]).to_csv("retry_failed_rics.csv", index=False)
    print(f"\n✓ Saved failed RICs to retry_failed_rics.csv")

print("\nDone!")

Analyzing existing data for incomplete RICs...
Columns in CSV: ['RIC', 'Date', 'price', 'tri', 'volume', 'cap', 'bid', 'ask']
Shape: (476713, 8)

Checking these columns for missing data: ['price', 'tri', 'volume', 'cap', 'bid', 'ask']

RICs with complete data: 13
RICs with missing data: 310

Incomplete RICs: ['AALB.AS', 'ABB.ST', 'ABI.BR', 'ABIO.PA^J22', 'ABNd.AS', 'ACCP.PA', 'ACKB.BR', 'AD.AS', 'ADP.PA', 'ADYEN.AS', 'AEGN.AS', 'AIR.PA', 'AIRF.PA', 'AIRP.PA', 'AKE.PA', 'AKZO.AS', 'ALFA.ST', 'ALFEN.AS', 'ALIVsdb.ST', 'ALLFG.AS']
... and 290 more

Missing data breakdown for incomplete RICs:
              RIC  price  tri  volume  cap  bid  ask  total_missing
91     EDF.PA^F23   1536   19    1536    0   15   17           3123
78    DAMA.PA^B23   1538   21    1538    0    4   19           3120
3     ABIO.PA^J22   1538   21    1538    0    0   20           3117
250   SEVI.PA^B22   1532   15    1532    0   14   14           3107
70    CNPP.PA^F22   1528   11    1528    0    9   10           3

C:\Users\Pranay Kumar\AppData\Local\Temp\ipykernel_12236\533499807.py:178:DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [45]:
pd.read_csv("attempt2.csv").isnull().sum()

RIC            0
Date           0
price     205811
tri          360
volume    205897
cap         2323
bid          494
ask          563
dtype: int64